In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import pandas as pd
import numpy as np

train_ori = pd.read_csv('/content/drive/MyDrive/부트캠프/store_train_transactions (1).csv')
test_ori = pd.read_csv('/content/drive/MyDrive/부트캠프/store_test_transactions (1).csv')

In [51]:
train_ori["구매일시"] = pd.to_datetime(train_ori["구매일시"])
test_ori["구매일시"] = pd.to_datetime(test_ori["구매일시"])

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [52]:
train_ori['웨딩성수기'] = train_ori['구매일시'].dt.month.isin([4, 5, 9, 10])

# 금액대별로 웨딩 성수기 기간의 구매 횟수
train_ori['웨딩성수기_저가구매'] = ((train_ori['웨딩성수기']) & (train_ori['구매가격'] <= 28000)).astype(int)
train_ori['웨딩성수기_중저가구매'] = ((train_ori['웨딩성수기']) & (train_ori['구매가격'] > 28000) & (train_ori['구매가격'] <= 58000)).astype(int)
train_ori['웨딩성수기_중고가구매'] = ((train_ori['웨딩성수기']) & (train_ori['구매가격'] > 58000) & (train_ori['구매가격'] <= 128000)).astype(int)
train_ori['웨딩성수기_고가구매'] = ((train_ori['웨딩성수기']) & (train_ori['구매가격'] > 128000)).astype(int)

# ID별로 금액대별 웨딩 성수기 구매 횟수, 웨딩시즌 전체 구매 횟수의 합 합계 계산
tmp = train_ori.groupby('ID').agg({
    '웨딩성수기_저가구매': 'sum',
    '웨딩성수기_중저가구매': 'sum',
    '웨딩성수기_중고가구매': 'sum',
    '웨딩성수기_고가구매': 'sum',
    '웨딩성수기': 'sum'
}).reset_index()

# 각 금액대별 구매 비율 계산
tmp['웨딩성수기_저가구매_비율'] = tmp['웨딩성수기_저가구매'] / tmp['웨딩성수기']
tmp['웨딩성수기_중저가구매_비율'] = tmp['웨딩성수기_중저가구매'] / tmp['웨딩성수기']
tmp['웨딩성수기_중고가구매_비율'] = tmp['웨딩성수기_중고가구매'] / tmp['웨딩성수기']
tmp['웨딩성수기_고가구매_비율'] = tmp['웨딩성수기_고가구매'] / tmp['웨딩성수기']


# 병합
train_ft = train_ori.merge(tmp, how='left', on="ID")
train_ft.head()


,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격,웨딩성수기_x,웨딩성수기_저가구매_x,웨딩성수기_중저가구매_x,웨딩성수기_중고가구매_x,웨딩성수기_고가구매_x,웨딩성수기_저가구매_y,웨딩성수기_중저가구매_y,웨딩성수기_중고가구매_y,웨딩성수기_고가구매_y,웨딩성수기_y,웨딩성수기_저가구매_비율,웨딩성수기_중저가구매_비율,웨딩성수기_중고가구매_비율,웨딩성수기_고가구매_비율
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700,True,0,0,1,0,7,11,5,11,34,0.205882,0.323529,0.147059,0.323529
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000,True,1,0,0,0,8,10,3,7,28,0.285714,0.357143,0.107143,0.250000
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937,True,0,1,0,0,18,5,6,0,29,0.620690,0.172414,0.206897,0.000000
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000,True,1,0,0,0,16,8,9,3,36,0.444444,0.222222,0.250000,0.083333
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000,True,0,0,0,1,0,0,2,1,3,0.000000,0.000000,0.666667,0.333333
